### Load the Libraries and import the data

In [2]:
import pandas as pd
import numpy as np
import sqlite3
import pandas_profiling
import rpy2
import rpy2.rinterface
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
import warnings # current version of seaborn generates a bunch of warnings that we'll ignore
warnings.filterwarnings("ignore")
import seaborn as sns
#import rpy2.ipython

In [2]:
#Read in the data from the file
df = pd.read_csv('C:/githubrepo/7331_Project/data/Employee_Compensation.csv')
print("Finished Loading Data.")

Finished Loading Data.


## Data Quality
Visualize the negatives as percentage (from Orig Data)
Job Family Code: 36 Unassigned
Job Family Code: 2451 with code of 0
Job Family: 2451 Untitled
Job Family: 36 Unassigned

Dept is 50% blank but can be fixed using reference https://data.sfgov.org/City-Management-and-Ethics/Reference-Department-Code-List/j2hz-23ps/data

In [3]:
#Save the dataframe into a new dataframe to clean
df_clean = df
#Keeps all of the Fiscal Year entries
df_clean = df_clean[df_clean['Year Type'] == 'Fiscal']
print(df_clean.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 334535 entries, 2 to 831991
Data columns (total 22 columns):
Year Type                  334535 non-null object
Year                       334535 non-null int64
Organization Group Code    334535 non-null int64
Organization Group         334535 non-null object
Department Code            305591 non-null object
Department                 213202 non-null object
Union Code                 334320 non-null float64
Union                      334320 non-null object
Job Family Code            334535 non-null object
Job Family                 334535 non-null object
Job Code                   334535 non-null object
Job                        334534 non-null object
Employee Identifier        334535 non-null int64
Salaries                   334535 non-null float64
Overtime                   334535 non-null float64
Other Salaries             334535 non-null float64
Total Salary               334535 non-null float64
Retirement                 334535 non

### Cleaning

In [4]:
#Get names of indexes for which column Year has value 2028
indexNames = df_clean.loc[df_clean['Year'] == 2028, 'Year'].index
# Delete these row indexes from dataFrame
df_clean.drop(indexNames, inplace = True)

In [5]:
#Get names of indexes for which column Job is Null
indexNames = df_clean.loc[df_clean['Job'] == np.NaN, 'Year'].index
# Delete these row indexes from dataFrame
df_clean.drop(indexNames, inplace = True)

In [ ]:
#Deletes the 1 row missing the Job
indexNames = df_clean.loc[df_clean['Job'].isnull()].index
df_clean.drop(indexNames, inplace = True)
df_clean.describe()

### Removes rows with negative values

In [7]:
#Delete Salaries values with negatives
indexNames = df_clean.loc[df_clean['Salaries'] < 0, 'Salaries'].index
df_clean.drop(indexNames, inplace = True)

In [8]:
#Delete Overtime values with negatives
indexNames = df_clean.loc[df_clean['Overtime'] < 0, 'Overtime'].index
df_clean.drop(indexNames, inplace = True)

In [9]:
#Delete Other Salaries values with negatives
indexNames = df_clean.loc[df_clean['Other Salaries'] < 0, 'Other Salaries'].index
df_clean.drop(indexNames, inplace = True)

In [10]:
#Delete Total Salary values with negatives
indexNames = df_clean.loc[df_clean['Total Salary'] < 0, 'Total Salary'].index
df_clean.drop(indexNames, inplace = True)

In [11]:
#Delete Retirement values with negatives
indexNames = df_clean.loc[df_clean['Retirement'] < 0, 'Retirement'].index
df_clean.drop(indexNames, inplace = True)

In [12]:
#Delete Health and Dental values with negatives
indexNames = df_clean.loc[df_clean['Health and Dental'] < 0, 'Health and Dental'].index
df_clean.drop(indexNames, inplace = True)

In [13]:
#Delete Other Benefits values with negatives
indexNames = df_clean.loc[df_clean['Other Benefits'] < 0, 'Other Benefits'].index
df_clean.drop(indexNames, inplace = True)

In [14]:
#Delete Total Benefits values with negatives
indexNames = df_clean.loc[df_clean['Total Benefits'] < 0, 'Total Benefits'].index
df_clean.drop(indexNames, inplace = True)

In [15]:
#Delete Total Compensation values with negatives
indexNames = df_clean.loc[df_clean['Total Compensation'] < 0, 'Total Compensation'].index
df_clean.drop(indexNames, inplace = True)

### Shows that there are no more negatives

In [ ]:
df_clean.describe()

### Fixes Union Code

In [16]:
#=========Union Code is by Job=========
#Information Systems Trainee - 21, "Prof & Tech Eng, Local 21"
#Recreation Facility Assistant - 790, "SEIU, Local 1021, Misc"
#Technology Expert II - 351, "Municipal Exec Assoc, Misc"
#Transportation Controller Trainee - NONE, "NONE"
df_clean.loc[(df_clean['Job'] == "Information Systems Trainee") & (df_clean['Union Code'].isnull()), 'Union Code'] = 21
df_clean.loc[(df_clean['Job'] == "Information Systems Trainee") & (df_clean['Union'].isnull()), 'Union'] = "Prof & Tech Eng, Local 21"
#===================================================================================================================================================
df_clean.loc[(df_clean['Job'] == "Recreation Facility Assistant") & (df_clean['Union Code'].isnull()), 'Union Code'] = 790
df_clean.loc[(df_clean['Job'] == "Recreation Facility Assistant") & (df_clean['Union'].isnull()), 'Union'] = "SEIU, Local 1021, Misc"
#===================================================================================================================================================
df_clean.loc[(df_clean['Job'] == "Technology Expert II") & (df_clean['Union Code'].isnull()), 'Union Code'] = 351
df_clean.loc[(df_clean['Job'] == "Technology Expert II") & (df_clean['Union'].isnull()), 'Union'] = "Municipal Exec Assoc, Misc"
#===================================================================================================================================================
df_clean.loc[(df_clean['Job'] == "Transportation Controller Trainee") & (df_clean['Union Code'].isnull()), 'Union Code'] = 0
df_clean.loc[(df_clean['Job'] == "Transportation Controller Trainee") & (df_clean['Union'].isnull()), 'Union'] = "None"


### Now see that there is very little missing data

In [5]:
# Check Missing Data
total = df_clean.isnull().sum().sort_values(ascending = False)
percent = (df_clean.isnull().sum() / df_clean.isnull().count() * 100).sort_values(ascending = False)
pd.concat([total, percent], axis = 1, keys = ['Total', 'Percent']).transpose()

,Department,Department Code,Total Compensation,Total Benefits,Year,Organization Group Code,Organization Group,Union Code,Union,Job Family Code,...,Job,Employee Identifier,Salaries,Overtime,Other Salaries,Total Salary,Retirement,Health and Dental,Other Benefits,Year Type
Total,121254.000000,28940.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Percent,36.347776,8.675216,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Fix Job Family Code and Job Family 

# WARNING! RUNS FOR A LONG TIME

In [18]:
#Job Family Code = Job Family. They are determined by Job Code

#All the jobs
jobs = df_clean['Job Code'].unique()
dfj = pd.DataFrame(data=jobs)
dfj.columns = ['Job Code']

#Gets all unique groupings of Job Family/Code and Job Code
df_jobstuff = pd.DataFrame(columns=['Job Code','Job Family','Job Family Code'])
for index, row in dfj.iterrows():
    t = df_clean[ (df_clean['Job Code'] == row['Job Code']) & ((df_clean['Job Family'] != "Unassigned") & (df_clean['Job Family'] != "Untitled"))]
    arg = t.groupby(['Job Code', 'Job Family', 'Job Family Code']).size().reset_index(name = 'Freq')
    if arg['Job Code'].count() > 1:
        arg.head()
    arg = arg[['Job Code', 'Job Family', 'Job Family Code']]
    df_jobstuff = df_jobstuff.append(arg)
    
print("Done")

Done


##### Does the actual fixing of the unassigned and unknown values

### WARNING! RUNS FOR A LONG TIME

In [19]:
df_clean = df_clean.reset_index(drop=True)

#Get a list of row ID's of ones missing stuff
errRowNums = df_clean.index[(df_clean['Job Family'] == "Unassigned") | (df_clean['Job Family'] == "Untitled")].tolist()
length = len(errRowNums)
for i in range(length): 
    jobname = df_clean.iloc[errRowNums[i]][10]
    r = df_jobstuff[df_jobstuff['Job Code'] == jobname]
    if r.empty:
        continue
    df_clean.loc[errRowNums[i], 'Job Family'] = r.iloc[0]['Job Family']
    df_clean.loc[errRowNums[i], 'Job Family Code'] = r.iloc[0]['Job Family Code']

print("Done!")


Done!


### Save the cleaned dataset 'df_clean' to a .csv file

In [24]:
df_clean.to_csv("EmpComp_Cleaned.csv", index = False, sep=',')
print("Done Saving File")

Done Saving File


# Still need to fix Department and Department Code columns

In [2]:
#Read in the data from the file
df = pd.read_csv('C:/githubrepo/7331_Project/data/EmpComp_Cleaned.csv')
print("Finished Loading Data.")
df_clean = df

Finished Loading Data.


### Fixing the empty Department values

# WARNING, TAKES A LONNNNNG TIME TO RUN

In [10]:
dptcode = df_clean['Department Code'].unique()
df_dc = pd.DataFrame(data=dptcode)
df_dc.columns = ['Department Code']
df_dc = df_dc[df_dc['Department Code'].notnull()]

In [12]:
#Gets all unique groupings of Job Family/Code and Job Code
df_dptstuff = pd.DataFrame(columns=['Department Code','Department'])
for index, row in df_dc.iterrows():
    t = df_clean[ (df_clean['Department Code'] == row['Department Code']) & (df_clean['Department'].notnull())]
    arg = t.groupby(['Department Code', 'Department']).size().reset_index(name = 'Freq')
    if arg['Department Code'].count() > 1:
        break
    arg = arg[['Department Code', 'Department']]
    df_dptstuff = df_dptstuff.append(arg)
    
print("Done")

Done


In [16]:
df_dptstuff.head()

,Department Code,Department
0,DPH,DPH Public Health
0,DPW,DPW GSA - Public Works
0,PUC,PUB Public Utilities Bureaus
0,ADM,ADM Gen Svcs Agency-City Admin
0,FIR,FIR Fire Department


In [21]:
df_clean = df_clean.reset_index(drop=True)

#Get a list of row ID's of ones missing stuff
errRowNums = df_clean.index[df_clean['Department'].isnull()].tolist()
length = len(errRowNums)
for i in range(length): 
    dptcode = df_clean.iloc[errRowNums[i]][4]
    r = df_dptstuff[df_dptstuff['Department Code'] == dptcode]
    if r.empty:
        continue
    df_clean.loc[errRowNums[i], 'Department'] = r.iloc[0]['Department']

print("Done!")

Done!


In [23]:
#Just backing it up for personal use
df_clean1 = df_clean

In [22]:
# Check Missing Data
total = df_clean.isnull().sum().sort_values(ascending = False)
percent = (df_clean.isnull().sum() / df_clean.isnull().count() * 100).sort_values(ascending = False)
pd.concat([total, percent], axis = 1, keys = ['Total', 'Percent']).transpose()

,Department,Department Code,Total Compensation,Total Benefits,Year,Organization Group Code,Organization Group,Union Code,Union,Job Family Code,...,Job,Employee Identifier,Salaries,Overtime,Other Salaries,Total Salary,Retirement,Health and Dental,Other Benefits,Year Type
Total,36936.000000,28940.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Percent,11.072142,8.675216,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
df_clean.to_csv("EmpComp_Cleaned.csv", index = False, sep=',')
print("Done Saving File")

Done Saving File


### Fixing the remaining Job Family and Job Family Code columns

In [9]:
df = pd.read_csv('C:/githubrepo/7331_Project/data/EmpComp_Cleaned.csv')
df_clean = df
print("Finished Loading Data.")

Finished Loading Data.


In [10]:
df_clean.loc[df_clean['Job Code'] == "0886", 'Job Family'] = "Management"
df_clean.loc[df_clean['Job Code'] == "0886", 'Job Family Code'] = "0900"
#======================================================================================================================================================
df_clean.loc[df_clean['Job Code'] == "381C", 'Job Family'] = "Legal and Count"
df_clean.loc[df_clean['Job Code'] == "381C", 'Job Family Code'] = "8100"
#======================================================================================================================================================
df_clean.loc[df_clean['Job Code'] == "274C", 'Job Family'] = "SF Superior Court"
df_clean.loc[df_clean['Job Code'] == "274C", 'Job Family Code'] = "SCRT"
#======================================================================================================================================================
df_clean.loc[df_clean['Job Code'] == "255C", 'Job Family'] = "SF Superior Court"
df_clean.loc[df_clean['Job Code'] == "255C", 'Job Family Code'] = "SCRT"

In [11]:
df_clean.to_csv("EmpComp_Cleaned.csv", index = False, sep=',')
print("Done Saving File")

Done Saving File
